## Steps
1. Import libraries
2. Create s3 bucket
3. Map train and test data in s3
4. Map the path of models in s3

### Import libraries

In [55]:
import sagemaker
import boto3   ## For creating and accessing s3 buckets
from sagemaker.session import s3_input, Session
from sagemaker.serializers import CSVSerializer

import pandas as pd
import numpy as np
import urllib
import os

### s3 Bucket creation

In [56]:
bucket_name = 'bankapplication71625347'
my_region = boto3.session.Session().region_name # To know the region to create s3 bucket in the same region
print(my_region)

us-west-2


In [57]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-west-2':
        location = {'LocationConstraint': my_region}   # Needed for locations other than us-east-1
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration = location)
    print('s3 bucket created successfully!!')
except Exception as e:
    print('s3 creation error: ', e)

s3 creation error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [58]:
# set output path for model
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://testapplication71625347/xgboost-as-a-built-in-algo/output


### Get Data, Split into train and test data, upload to s3 bucket

In [59]:
# Get the data
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean_data.csv")
    print('Success!! Data download complete')
except Exception as e:
    print('Data load error: ',e)
try:
    model_data = pd.read_csv('./bank_clean_data.csv',index_col=0)
    print('Success: Data loaded into dataframe')
except Exception as e:
    print('Data load error: ',e)

Success!! Data download complete
Success: Data loaded into dataframe


In [60]:
# Split data into test and train data
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1625), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [61]:
# Save train and test data into buckets
# First the train data
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/train'.format(bucket_name, prefix),content_type='csv')

In [62]:
# Now, the test data
pd.concat([test_data['y_yes'], test_data.drop(['y_no','y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building and training xGBoost in-built algorithm

In [63]:
# automatically looks for the XGBoost image URI and builds an XGBoost container.
container = sagemaker.image_uris.retrieve('xgboost',
                          boto3.Session().region_name, 
                          version='1.2-1')
print(container)

246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1


In [64]:
# hyperparams
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [65]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [66]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

2022-03-17 16:42:24 Starting - Starting the training job...
2022-03-17 16:42:54 Starting - Launching requested ML instancesProfilerReport-1647535344: InProgress
.........
2022-03-17 16:44:14 Starting - Preparing the instances for training......
2022-03-17 16:45:16 Downloading - Downloading input data
2022-03-17 16:45:16 Training - Downloading the training image...
2022-03-17 16:45:55 Training - Training image download completed. Training in progress..[2022-03-17 16:45:52.564 ip-10-0-177-151.us-west-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO

### Deploy model

In [70]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'Number of instances across active endpoints' is 2 Instances, with current utilization of 2 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

### Prediction of test data

In [ ]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

### Deleting endpoints

In [72]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-west-2:730224035312:endpoint/sagemaker-xgboost-2022-03-17-16-50-08-687".